In [148]:
import numpy as np

import math

from matplotlib.pyplot import figure
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This notebook simulates the rough Heston model as introduced by Guennoun, Jacquier, Roome and Shi in <cite data-cite="guennoun2018asymptotic">Guennoun, Hamza, et al. "Asymptotic behavior of the fractional Heston model." SIAM Journal on Financial Mathematics 9.3 (2018): 1017-1045.</cite>

In [248]:
n = 1000 # Number of timesteps
M = 1 # Number of simulations
rho = 1 # Correlation between the normal distributions dzeta and eta
T = 2 # Total time
alpha = 0.25 # Alpha as in G^\alpha

# Some volatility parameters
kappa = 10
theta = 1
ksi = 1
Y_0 = 10 # initial value of Y
X_0 = 1 # initial value of X, the stock price 

In [249]:
dzeta = np.random.normal(0.0, 1.0, size=(n,))

In [250]:
Y = np.zeros((n,))
Delta_Y = np.zeros((n,))

In [251]:
def g(t):
    return t**alpha
g_vector = np.fromfunction(lambda i: g(T*i/n), shape=(n,), dtype=int)

In [252]:
def b(y):
    return kappa * (theta-y)
def a(y):
    return ksi * math.sqrt(y)

The Notebook first simulates the volatility

In [253]:
sqrtn = math.sqrt(n)
Y[0] = Y_0
for i in range(1, n):
    Y[i] = Y[i-1] + (T/n)*b(Y[i-1]) + (T/sqrtn)*a(Y[i-1])*dzeta[i-1]
    Delta_Y[i] = Y[i] - Y[i - 1]

Using Fast Fourier Transform (FFT), the volatility is made rough. For more details, see <cite data-cite="horvath2017functional">Horvath, Blanka, Antoine Jack Jacquier, and Aitor Muguruza. "Functional central limit theorems for rough volatility." Available at SSRN 3078743 (2017)</cite>

In [254]:
fourier_transformed_G_alpha = np.fft.rfft(g_vector)*np.fft.rfft(Delta_Y) 
G_alpha_Y = np.fft.irfft(fourier_transformed_G_alpha)

For some reason ^ produces a G which is negative and doesn't start at 0.

TODO: 
- resolve that issue.
- check all code for general mistakes

<!--bibtex

@article{guennoun2018asymptotic,
  title={Asymptotic behavior of the fractional Heston model},
  author={Guennoun, Hamza and Jacquier, Antoine and Roome, Patrick and Shi, Fangwei},
  journal={SIAM Journal on Financial Mathematics},
  volume={9},
  number={3},
  pages={1017--1045},
  year={2018},
  publisher={SIAM}
}

@article{horvath2017functional,
  title={Functional central limit theorems for rough volatility},
  author={Horvath, Blanka and Jacquier, Antoine Jack and Muguruza, Aitor},
  journal={Available at SSRN 3078743},
  year={2017}
}

-->

In [255]:
X = np.zeros((M, n))
for j in range(M):
    # Create correlated Brownian motion
    randoms = np.random.normal(0.0, 1.0, size=(n,))
    eta = rho*dzeta + math.sqrt(1 - rho**2) * randoms
    X[j, 0] = X_0
    for i in range(1, n):
        X[j, i] = X[j, i-1] - (1/2)*(T/n)*G_alpha_Y[i-1] + math.sqrt((T/n) * G_alpha_Y[i-1]) * eta[i-1]

ValueError: math domain error

In [256]:
G_alpha_Y

array([-10.47839006, -10.44031277, -10.10675165,  -9.74457933,
        -9.28062127,  -9.26410326,  -9.41559617,  -9.01356405,
        -9.03303968,  -8.9512599 ,  -8.63676252,  -8.89580305,
        -8.93771944,  -9.08687919,  -8.78777366,  -8.72717884,
        -8.45016731,  -8.47327024,  -8.73097385,  -8.35472444,
        -8.37451882,  -8.38418479,  -8.24653895,  -8.22271354,
        -7.96793995,  -7.77260508,  -7.45989756,  -7.56646305,
        -7.41311561,  -7.51080562,  -7.47291826,  -7.47790363,
        -7.54219863,  -7.32986607,  -7.59949268,  -7.45102729,
        -7.60570868,  -7.42029048,  -7.18351674,  -7.25282412,
        -7.07742049,  -7.21829125,  -7.16618601,  -7.16530416,
        -7.06721772,  -7.04614516,  -6.62529012,  -6.66310992,
        -6.68882983,  -6.78708272,  -6.87167238,  -6.77725878,
        -6.79851402,  -6.68061806,  -6.56754837,  -6.48987719,
        -6.53806337,  -6.44165828,  -6.16272109,  -6.10460551,
        -5.93930528,  -6.09468389,  -5.80752278,  -5.90